In [1]:
import os
import scipy.io as sio
import numpy as np
from PIL import Image, ImageFilter

In [2]:
#general file access
src_folder = ".../IMAGE-SELECTION/full-selection/"
easy_folder = ".../IMAGE-SELECTION/easy10/"
difficult_folder = ".../IMAGE-SELECTION/difficult100/"
dest_folder = ".../IMAGE-SELECTION/processed/"
annotation_folder = ".../pascal-parts/Annotations_Part/"
highlight_folder=".../IMAGE-SELECTION/highlights/"
types=os.listdir(src_folder)


In [3]:
#single image getter function
#usable indices: starting with 0
#easy: 0-9
#difficult:0-99
#all: aeroplane-385; bird-365; boat-242; bottle-116; car-370; chair-197; 
    #dog-664; horse-201; person-902; tvmonitor-134
def get_image(category,difficulty,number):
    if number<0:
        print("only positive index allowed")
    else:
        if difficulty=='easy':
            available=os.listdir(easy_folder+category)
            if number<len(available):
                return Image.open(easy_folder+category+"/"+available[number])
            else:
                print("input number higher than number of available images")
        elif difficulty=='difficult':
            available=os.listdir(difficult_folder+category)
            if number<len(available):
                return Image.open(difficult_folder+category+"/"+available[number]) 
            else:
                print("input number higher than number of available images")
        elif difficulty=='all':
            available=os.listdir(src_folder+category)
            if number<len(available):
                return Image.open(src_folder+category+"/"+available[number])
            else:
                print("input number higher than number of available images")
        else:
            print("please put a valid difficulty - easy,difficult,all - in get_image")

In [4]:
#function to get image number from simple indexing
#usable indices: starting with 0
#easy: 0-9
#difficult:0-99
#all: aeroplane-385; bird-365; boat-242; bottle-116; car-370; chair-197; 
    #dog-664; horse-201; person-902; tvmonitor-134
def get_imagenumber(category,difficulty,number):
    if number<0:
        print("only positive index allowed")
    else:
        if difficulty=='easy':
            available=os.listdir(easy_folder+category)
            if number<len(available):
                return available[number].rstrip('.jpg')
            else:
                print("input number higher than number of available images")
        elif difficulty=='difficult':
            available=os.listdir(difficult_folder+category)
            if number<len(available):
                return available[number].rstrip('.jpg')
            else:
                print("input number higher than number of available images")
        elif difficulty=='all':
            available=os.listdir(src_folder+category)
            if number<len(available):
                return available[number].rstrip('.jpg')
            else:
                print("input number higher than number of available images")
        else:
            print("please put a valid difficulty - easy,difficult,all - in get_image")

In [5]:
#darken single image background
def darken_background(imagenumber,image_class,factor=0.0):
    #load annotation and access mask data
    mat_content = sio.loadmat(annotation_folder+imagenumber+'.mat')
    annotation = mat_content['anno'][0,0]
    objects = annotation[1][0]
    #collect object annotations that match current category
    object_pos=[]
    for obj in objects:
        if obj[0][0]==image_class:
            object_pos.append(obj[2])
    #create combined mask of >not to darken< pixels from interesting object masks
    outline=np.zeros((333,500))
    for pos_field in object_pos:
        if pos_field.shape[0]>pos_field.shape[1]:
            pos_field=np.rot90(pos_field,-1)
        #as images are cropped to the fixed size, annotations have to be, too
        outline+=pos_field[:333,:]
        
    #image access    
    im = Image.open(src_folder+image_class+"/"+imagenumber+".jpg")
    pixels = im.convert('RGB')
    imarray = np.array(pixels)
    
    #create lightness weight matrix and apply to image channels
    weights= np.where(outline<1,factor,1.0)
    imarray[:,:,0]=imarray[:,:,0]*weights
    imarray[:,:,1]=imarray[:,:,1]*weights
    imarray[:,:,2]=imarray[:,:,2]*weights
    #return changed image
    return Image.fromarray(imarray)   

In [6]:
#reduce contrast of a single image
def reduce_contrast(imagenumber,image_class,factor=0.4):
    #load annotation and access mask data
    mat_content = sio.loadmat(annotation_folder+imagenumber+'.mat')
    annotation = mat_content['anno'][0,0]
    objects = annotation[1][0]

    #collect object annotations that match current category
    object_pos=[]
    for obj in objects:
        if obj[0][0]==image_class:
            object_pos.append(obj[2])

    #create combined mask of pixels where contrast should stay normal from interesting object masks
    outline=np.zeros((333,500))
    for pos_field in object_pos:
        if pos_field.shape[0]>pos_field.shape[1]:
            pos_field=np.rot90(pos_field,-1)
        #as images are cropped to the fixed size, annotations have to be, too
        outline+=pos_field[:333,:]

    #sub function to apply on the points
    def contrast_reduction(pix):
        return 128 + factor * (pix - 128)

    #image access
    im = Image.open(src_folder+image_class+"/"+imagenumber+".jpg")
    pixels = im.convert('RGB')
    imarray = np.array(pixels)
    #image with reduced contrast to take pixels from where background is
    reducedarray = np.array(im.point(contrast_reduction).convert('RGB'))

    #decision matrix for 'where' function contains information where background is
    background= np.where(outline<1,1,0)
    imarray[:,:,0]=np.where(background,reducedarray[:,:,0],imarray[:,:,0])
    imarray[:,:,1]=np.where(background,reducedarray[:,:,1],imarray[:,:,1])
    imarray[:,:,2]=np.where(background,reducedarray[:,:,2],imarray[:,:,2])
    return Image.fromarray(imarray)

In [7]:
#blurring the background of single image
def background_blurring(imagenumber,image_class,factor=4):
    #load annotation and access mask data
    mat_content = sio.loadmat(annotation_folder+imagenumber+'.mat')
    annotation = mat_content['anno'][0,0]
    objects = annotation[1][0]

    #collect object annotations that match current category
    object_pos=[]
    for obj in objects:
        if obj[0][0]==image_class:
            object_pos.append(obj[2])
    #create combined mask of pixels where no blurring should take place from interesting object masks
    outline=np.zeros((333,500))
    for pos_field in object_pos:
        if pos_field.shape[0]>pos_field.shape[1]:
            pos_field=np.rot90(pos_field,-1)
        #as images are cropped to the fixed size, annotations have to be, too
        outline+=pos_field[:333,:]

    #image access
    im = Image.open(src_folder+image_class+"/"+imagenumber+".jpg")
    pixels = im.convert('RGB')
    imarray = np.array(pixels)
    #blurred image to take pixels from where background is
    blurredarray = np.array((im.filter(ImageFilter.GaussianBlur(factor))).convert('RGB'))

    #decision matrix for 'where' function contains information where background is
    background= np.where(outline<1,1,0)
    imarray[:,:,0]=np.where(background,blurredarray[:,:,0],imarray[:,:,0])
    imarray[:,:,1]=np.where(background,blurredarray[:,:,1],imarray[:,:,1])
    imarray[:,:,2]=np.where(background,blurredarray[:,:,2],imarray[:,:,2])
    return Image.fromarray(imarray)

In [8]:
#get image outline for single image
def image_outline(imagenumber,image_class,factor=3):
    #load annotation and access mask data
    mat_content = sio.loadmat(annotation_folder+imagenumber+'.mat')
    annotation = mat_content['anno'][0,0]
    objects = annotation[1][0]

    #collect object annotations that match current category
    object_pos=[]
    for obj in objects:
        if obj[0][0]==image_class:
            object_pos.append(obj[2])
    #create combined outlines from interesting object masks
    outline=np.zeros((333,500))
    for pos_field in object_pos:
        if pos_field.shape[0]>pos_field.shape[1]:
            pos_field=np.rot90(pos_field,-1)
        #as images are cropped to the fixed size, annotations have to be, too
        location=pos_field[:333,:]
        #check for background pixels near object pixels-> outline
        for j in range(0,333):
            for i in range(0,500):
                if location[j,i]==1:
                    if np.any(location[max(0,j-factor):min(333,j+factor+1),max(0,i-factor):min(500,i+factor+1)]==0):
                        outline[j,i]=1
    #outline is white, everything else black
    return Image.fromarray(outline*255).convert('RGB')

In [9]:
#segment single image
def segment_objects(imagenumber,image_class):
    #load annotation and access mask data
    mat_content = sio.loadmat(annotation_folder+imagenumber+'.mat')
    annotation = mat_content['anno'][0,0]
    objects = annotation[1][0]

    #collect object annotations that match current category 
    object_pos=[]
    for obj in objects:
        if obj[0][0]==image_class:
            object_pos.append(obj[2])

    #image access
    im = Image.open(src_folder+image_class+"/"+imagenumber+".jpg")
    pixels = im.convert('RGB')
    imarray = np.array(pixels)

    #outlines for 3 color channels separately
    outlinesa=[]
    outlinesb=[]
    outlinesc=[]
    #create matrix with number of objects annotated in each pixel (to divide corresponding added colors by that number in the end)
    overlap=np.zeros((333,500))
    for pos_field in object_pos:
        if pos_field.shape[0]>pos_field.shape[1]:
            pos_field=np.rot90(pos_field,-1)
        #as images are cropped to the fixed size, annotations have to be, too
        location=pos_field[:333,:]
        #when object of interest is inside current annotation matrix compute mean color of respective pixels
        #and put those at the position of the object
        if np.sum(location)!=0:
            midval = np.sum(np.where(np.expand_dims(location,2),imarray,np.zeros((333,500,3))),axis=(0,1))/np.sum(location)
            outlinesa.append(location*midval[0])
            outlinesb.append(location*midval[1])
            outlinesc.append(location*midval[2])
            overlap+=location
    
    #map of background pixels
    backposition=1-(np.minimum(overlap,np.ones((333,500))))
    #number of background pixels
    backcount=np.sum(backposition)
    #when there are pixels that don't belong to an object (visible background)
    if backcount!=0:
        #compute colorchannels for the background pixels (where no interesting object was annotated)             
        backgrounda=np.where(overlap==0,imarray[:,:,0],np.zeros((333,500)))
        backgroundb=np.where(overlap==0,imarray[:,:,1],np.zeros((333,500)))
        backgroundc=np.where(overlap==0,imarray[:,:,2],np.zeros((333,500)))


        #compute mean color of background
        backcolora=np.sum(backgrounda)/backcount
        backcolorb=np.sum(backgroundb)/backcount
        backcolorc=np.sum(backgroundc)/backcount   
    else:
        backcolora=0
        backcolorb=0
        backcolorc=0
        
    #number of objects that correspond to one pixel with their mean color
    #replace 0 with -1 so that division by 0 is avoided, even though values on those positions are never used
    divisor=np.where(overlap==0,np.ones((333,500))*-1,overlap)
    #compute mean color for pixels where objects are
    coloredobjectsa =np.where(overlap>0,sum(outlinesa)/divisor,np.zeros((333,500)))
    coloredobjectsb =np.where(overlap>0,sum(outlinesb)/divisor,np.zeros((333,500)))
    coloredobjectsc =np.where(overlap>0,sum(outlinesc)/divisor,np.zeros((333,500)))    

    #combine background and object pixels to image
    channela=np.where(overlap==0,np.ones((333,500))*backcolora,coloredobjectsa)
    channelb=np.where(overlap==0,np.ones((333,500))*backcolorb,coloredobjectsb)
    channelc=np.where(overlap==0,np.ones((333,500))*backcolorc,coloredobjectsc)

    #combine color channels to image
    imarray =np.stack((channela,channelb,channelc),axis=-1)

    #operations require casting to uint8
    return Image.fromarray(imarray.astype('uint8'))

In [10]:
#segment single image including object parts
def segment_parts(imagenumber,image_class):
    #load annotation and access mask data
    mat_content = sio.loadmat(annotation_folder+imagenumber+'.mat')
    annotation = mat_content['anno'][0,0]
    objects = annotation[1][0]
    
    #collect object annotations that match current category 
    part_pos=[]
    for obj in objects:
        if obj[0][0]==image_class:
            part_pos.append(obj[3])
    #some categories do not have part segmentation annotations and return just the object segmentation result        
    if (part_pos[0].shape==(0,0)):
        return segment_objects(imagenumber,image_class)
    #image access
    im = Image.open(src_folder+image_class+"/"+imagenumber+".jpg")
    pixels = im.convert('RGB')
    imarray = np.array(pixels)

    #outlines for 3 color channels separately
    outlinesa=[]
    outlinesb=[]
    outlinesc=[]
    #create matrix with number of parts annotated in each pixel (to divide corresponding added colors by that number in the end)
    overlap=np.zeros((333,500))
    for part in part_pos:
        #some rare image annotations strangely contain empty part annotations, those have to be ignored
        if len(part)!=0:
            for pos_field in part[0]:
                part_array=pos_field[1]
                #rotate annotation matrix if portrait shaped
                if part_array.shape[0]>part_array.shape[1]:
                    part_array=np.rot90(part_array,-1)
                #as images are cropped to the fixed size, annotations have to be, too
                location=part_array[:333,:]
                #when a part of the object of interest is inside current annotation matrix compute mean color 
                #of respective pixels and put those at the position of the part
                if np.sum(location)!=0:
                    midval = np.sum(np.where(np.expand_dims(location,2),imarray,np.zeros((333,500,3))),axis=(0,1))/np.sum(location)
                    outlinesa.append(location*midval[0])
                    outlinesb.append(location*midval[1])
                    outlinesc.append(location*midval[2])
                    overlap+=location

    
    ##add object outlines for pixels that are not in any part of it (e.g. monitor frames)
    
    #collect object annotations that match current category 
    object_pos=[]
    for obj in objects:
        if obj[0][0]==image_class:
            object_pos.append(obj[2])
            
    #create matrix with object pixels that are not inside any of their parts
    outeroverlap=np.zeros((333,500))
    for pos_field in object_pos:
        if pos_field.shape[0]>pos_field.shape[1]:
            pos_field=np.rot90(pos_field,-1)
        #as images are cropped to the fixed size, annotations have to be, too
        location=pos_field[:333,:]
        #annotation of complete object only useful for pixels that are not in a part
        remaining=np.maximum(np.zeros((333,500)),location-overlap)
        #when object of interest is inside current annotation matrix compute mean color of object pixels
        #and put those at the remaining pixels of the object
        if np.sum(location)!=0:
            midval = np.sum(np.where(np.expand_dims(location,2),imarray,np.zeros((333,500,3))),axis=(0,1))/np.sum(location)
            outlinesa.append(remaining*midval[0])
            outlinesb.append(remaining*midval[1])
            outlinesc.append(remaining*midval[2])
            outeroverlap+=remaining
    
    #total pixels that are annotated by either parts or objects (of interesting class)
    overlap+=outeroverlap
                
    #map of background pixels
    backposition=1-(np.minimum(overlap,np.ones((333,500))))
    #number of background pixels
    backcount=np.sum(backposition)
    if backcount!=0:
        #compute colorchannels for the background pixels (where no interesting object was annotated)             
        backgrounda=np.where(overlap==0,imarray[:,:,0],np.zeros((333,500)))
        backgroundb=np.where(overlap==0,imarray[:,:,1],np.zeros((333,500)))
        backgroundc=np.where(overlap==0,imarray[:,:,2],np.zeros((333,500)))

        #compute mean color of background
        backcolora=np.sum(backgrounda)/backcount
        backcolorb=np.sum(backgroundb)/backcount
        backcolorc=np.sum(backgroundc)/backcount
    else:
        backcolora=0
        backcolorb=0
        backcolorc=0
    
    #number of objects that correspond to one pixels with their mean color
    #replace 0 with -1 so that division by 0 is avoided, even though values on those positions are never used
    divisor=np.where(overlap==0,np.ones((333,500))*-1,overlap)
    #compute mean color for pixels where objects are
    coloredobjectsa =np.where(overlap>0,sum(outlinesa)/divisor,np.zeros((333,500)))
    coloredobjectsb =np.where(overlap>0,sum(outlinesb)/divisor,np.zeros((333,500)))
    coloredobjectsc =np.where(overlap>0,sum(outlinesc)/divisor,np.zeros((333,500)))    

    #combine background and object pixels to image
    channela=np.where(overlap==0,np.ones((333,500))*backcolora,coloredobjectsa)
    channelb=np.where(overlap==0,np.ones((333,500))*backcolorb,coloredobjectsb)
    channelc=np.where(overlap==0,np.ones((333,500))*backcolorc,coloredobjectsc)

    #combine color channels to image
    imarray =np.stack((channela,channelb,channelc),axis=-1)

    #operations require casting to uint8
    return Image.fromarray(imarray.astype('uint8'))

In [11]:
#segment single image with binary output: black or white
def segment_binary(imagenumber,image_class):
    #load annotation and access mask data
    mat_content = sio.loadmat(annotation_folder+imagenumber+'.mat')
    annotation = mat_content['anno'][0,0]
    objects = annotation[1][0]

    #collect object annotations that match current category 
    object_pos=[]
    for obj in objects:
        if obj[0][0]==image_class:
            object_pos.append(obj[2])

    #create positiv-zero map for object locations
    overlap=np.zeros((333,500))
    for pos_field in object_pos:
        if pos_field.shape[0]>pos_field.shape[1]:
            pos_field=np.rot90(pos_field,-1)
        location=pos_field[:333,:]
        #when object of interest is inside current annotation matrix add points to positive area
        if np.sum(location)!=0:
            overlap+=location

    shape=np.where(overlap==0,0,255)
    #create image from positive-zero map
    imarray =np.stack((shape,shape,shape),axis=-1)
    #type has to be specified for Image casting to work
    return Image.fromarray(imarray.astype('uint8'))

In [15]:
#put out every version of 2 easy images from every category in the highlights folder as examples
for typee in types:
    print(typee)
    for i in range(6,8):
        get_image(typee,'easy',i).save(highlight_folder+typee+str(i%2+1)+"original.jpg",quality=100)
        background_blurring(get_imagenumber(typee,'easy',i),typee).save(highlight_folder+typee+str(i%2+1)+"blurred.jpg",quality=100)
        darken_background(get_imagenumber(typee,'easy',i),typee).save(highlight_folder+typee+str(i%2+1)+"darkened.jpg",quality=100)
        reduce_contrast(get_imagenumber(typee,'easy',i),typee).save(highlight_folder+typee+str(i%2+1)+"contrast.jpg",quality=100)
        image_outline(get_imagenumber(typee,'easy',i),typee).save(highlight_folder+typee+str(i%2+1)+"outline.jpg",quality=100)
        segment_binary(get_imagenumber(typee,'easy',i),typee).save(highlight_folder+typee+str(i%2+1)+"binary.jpg",quality=100)
        segment_objects(get_imagenumber(typee,'easy',i),typee).save(highlight_folder+typee+str(i%2+1)+"seg-objects.jpg",quality=100)
        segment_parts(get_imagenumber(typee,'easy',i),typee).save(highlight_folder+typee+str(i%2+1)+"seg-parts.jpg",quality=100)
        

    

In [16]:
#apply all image modifications on all images in a category
def process_all(typee):
    current_folder_items = os.listdir(src_folder+typee+"/")
    for image in current_folder_items:
        imagenumber=image.rstrip('.jpg')
        background_blurring(imagenumber,typee).save(dest_folder+"background-blurring/"+typee+"/"+imagenumber+".jpg",quality=100)
        darken_background(imagenumber,typee).save(dest_folder+"background-dark/"+typee+"/"+imagenumber+".jpg",quality=100)
        reduce_contrast(imagenumber,typee).save(dest_folder+"background-contrast/"+typee+"/"+imagenumber+".jpg",quality=100)
        image_outline(imagenumber,typee).save(dest_folder+"outline-only/"+typee+"/"+imagenumber+".jpg",quality=100)
        segment_binary(imagenumber,typee).save(dest_folder+"segment-binary/"+typee+"/"+imagenumber+".jpg",quality=100)
        segment_objects(imagenumber,typee).save(dest_folder+"segment-object/"+typee+"/"+imagenumber+".jpg",quality=100)
        segment_parts(imagenumber,typee).save(dest_folder+"segment-parts/"+typee+"/"+imagenumber+".jpg",quality=100)



In [17]:
for typee in types:
    %time process_all(typee)

CPU times: user 1min 22s, sys: 1.46 s, total: 1min 24s
Wall time: 1min 24s
CPU times: user 52.1 s, sys: 2.27 s, total: 54.4 s
Wall time: 54.7 s
CPU times: user 55.8 s, sys: 1.77 s, total: 57.5 s
Wall time: 58 s
CPU times: user 1min 33s, sys: 1.76 s, total: 1min 35s
Wall time: 1min 36s
CPU times: user 1min 7s, sys: 635 ms, total: 1min 8s
Wall time: 1min 8s
CPU times: user 1min 2s, sys: 987 ms, total: 1min 3s
Wall time: 1min 3s
CPU times: user 1min 22s, sys: 3.53 s, total: 1min 25s
Wall time: 1min 26s
CPU times: user 44.2 s, sys: 1.49 s, total: 45.7 s
Wall time: 46 s
CPU times: user 54 s, sys: 2.52 s, total: 56.6 s
Wall time: 56.8 s
CPU times: user 1min 16s, sys: 2.21 s, total: 1min 19s
Wall time: 1min 19s
